# Identification of dynamic model parameters

## Dynamic modeling

(Notation mostly from here http://www.scholarpedia.org/article/Robot_dynamics)

__Forward dynamics__:

Given joint torques/forces, find the resulting velocities and accelerations.

$$\ddot{q} = F(q, \dot{q}, \tau, f_{ext})$$
with:  
* $F$ - some function
* $q$ - Joint positions in configuration space ($\in \mathbb{R}^n$, generalized coordinates)
* $f_\text{ext}$ - contact forces (at end-effectors)
* $\tau$ - joint torques

The forward dynamics can be solved numerically with the ABA algorithm.

__Inverse dynamics__:

Given the position, velocity and acceleration of joint trajectories, find the necessary
torques/forces of the joints. (Sometimes simply called equations of motion, joint-space form)

$$\tau = H(q)\ddot{q}+c(q,\dot{q}, f_\text{ext})$$
with:  
* $H$ - joint-space inertia (or "mass") matrix $\in \mathbb{R}^{6+n_\text{dof} \times 6+n_\text{dof}}$
* $c$ - joint-space bias force (the force which is needed to produce zero acceleration)
  
More detailed, the inverse dynamics with bias forces broken down:  
$$\tau = H(q)\ddot{q}+C(q,\dot{q})\dot{q} + g(q) + J(q)^T f_\text{ext}$$
with:  
* $C\dot{q}$ - Coriolis and centrifugal forces ($n \times 1$, C $n \times n$)
* $g$ - Gravity terms ($n \times 1$)
* $J$ - Jacobian of the end-effector(s)
* $f_\text{ext}$ - Wrench excerted by external environment on link
* (friction is missing, for that see p.228 Khalil)

The inverse dynamics problem can be solved with the RNEA algorithm (which can also give $c(q,\dot{q}, f_\text{ext})$).

Regressor form:

$\tau = Y(\theta, \dot{\theta}, \ddot{\theta}) x$, with:  
* $Y$ - observation or estimated observation matrix or dynamics regressor ($n_\text{dof} \times 10n_\text{links}$)
* $x$ - inertial parameter vector ($10n_\text{links} \times 1$)

The inertia parameters usually consist of mass (1 param), location of center of mass (3) and rotational inertia in each axis (6), so **10** parameters for each link.
Note: they have different units, so might have to be weighted for some calculations, e.g. variance.

## Identification

We're looking to identify the inertial parameters vector $x$ (mass, com, inertia for each joint).

In CoDyCo/iDynTree notation (https://github.com/robotology/idyntree/blob/master/doc/dcTutorialCpp.md):  
"For floating-base dynamics, the dynamics regressor is a $(6+\text{dofs})\times(10 \cdot \text{nrOfLinks})$ matrix $Y$ such that: $Y \pi = M(q) \frac{d(v)}{dt} + C(q,v)v + g(q)$ with $M(q)$, $C(q,v)$ and $g(q)$ defined in http://wiki.icub.org/codyco/dox/html/dynamics_notation.html. The $\pi$ vector is a $10 \cdot \text{nrOfLinks}$ inertial parameters vector, such that the elements of the vector from the $((i-1) \cdot 10)$-th to the $((i \cdot 10) - 1))$-th belong to the i-th link. For more details on the inertial parameters vector, check https://hal.archives-ouvertes.fr/hal-01137215/document".

iDynTree can calculate the dynamics regressor $Y$, which is dependent on the joint-link tree and on the system state ($q, \dot{q}, \ddot{q}$). For Walk-Man, the contact forces with the ground at the feet can be measured. For fixed base dynamics, the only force is coming through the constrained base link (for which the Jacobian is $J = [I \quad 0]$ and the contact forces vanish).

When torques and/or contact forces are known and Y is retrieved through iDynTree (also $J^T$ of each contact link), we can calculate 

$$ Yx = \tau - \sum_{l \in L} J_{l}^T f_l^{ext} $$
$$ x = Y^{-1}\tau - Y^{-1}\sum_{l \in L} J_{l}^T f_l^{ext} $$
(using pseudoinverse instead of real inverse)
$$ \tilde{x} = Y^{+}\tau - Y^{+}\sum_{l \in L} J_{l}^T f_l^{ext} $$

(Y is only dependent on motion data and model, and can be calculated without knowing x)

In order to do identification of $x$ without having to know the contact forces, the HyQ identification code from Andrea and Guido uses a null-space multiplication to eliminate the contact forces (they don't have sensors in the feet).
 
$$ Yx = \tau - J^T f_\text{ext} $$
$$ N(J^T)Yx = N(J^T)\tau - \underbrace{N(J^T)J^T f_\text{ext}}_{=0}$$
$$ x = [N(J^T)Y]^{-1}N(J^T)\tau $$

If only contact forces are known, there is also the possibility to just look at the upper equations of the matrix formulation and leave out the joint torques (see: Ayusawa, Venture: "Identifiability and identification of inertial parameters using the underactuated base-link dynamics for legged multibody systems").

Before doing the inversion, we need to reduce the standard parameters to the actually identifiable base parameters by reducing $Y$ to the linearly independent columns, e.g. using Gautier algorithm, SVD or QR. Otherwise the pseudoinverse least squares approximation will be inaccurate. If $Y$ is not of full rank (i.e. columns not reduced), a damped least-squares method is recommended by Siciliano, et al (p.282). There is computeFloatingBaseIdentifiableSubspace() in iDynTree. The resulting basis $B$ of the base parameter subspace can be used to project from the standard regressor to the base regressor, i.e. $Y_\text{base} = Y_\text{std}B$.
After obtaining the base parameters $\tilde{x}$, we need to project them back to the standard basis in order to use them for e.g. simulation models or modifying the URDF file. When using a symbolic regressor, both can stay in the base subspace for e.g. torque prediction.
Using the transpose of $B$ (transpose is here equal to pseudoinverse because the columns are already linearly independent), we can calculate $x_\text{std} = B^T x_\text{base}$.
This way, an estimation of mean values is achieved to get the unknown distribution of one column to multiple ones.
In any case, those are the columns that ideally don't have an impact to the result anyway (or a very small one).

The inversion assumes that the equations are linear in the unknown parameters. This is however only true for Recursive Newton-Euler if the parameters are given . The inertia values/tensors need to be given in relation to the link frame (URDF uses relative to the COM). The COM needs to be given as first moment of mass. See also Khosla, 1985.

## Retrieving torque measurements

The torque measurements should measure the deflection between link $i$ and child link $i+1$. For the position, velocity and acceleration, it is likely necessary to differentiate the velocity and possibly also the position depending on what measurements are available. Since differentiation has the effect of high-pass filtering and therefore adding new noise, the data can be low-pass filtered (e.g. with an appropriately designed butterworth filter). This however can introduce phase-shift. Another option is to assume a curve shape model, fit it to a
few data points and differentiate it analytically, it is also possible to use simple median filtering or RANSAC for additional elimation of outliers.

It is often done in previous work to use multiple experimental setups with separate body parts (going up the chain) while fixating the rest of the body to improve the estimation accuracy. For the iCub, the arms were "cut" at the position of the 6 DOF F/T sensors so that the measurement gives the contact forces for a fixed base scenario. The limbs should move with a standardized trajectory excitation that highlights all the influencing columns and has sufficient velocities, accelerations and forces appearing. The dynamic equations don't include friction or other non-linear effects. We can get the Jacobian(s) from iDynTree.

Using $N$ measurement samples at time instants $t_1 \dots t_N$, the regressor matrix is extended by v-stacking regressors for each measurement and the same for the torque vectors.

$$\begin{bmatrix} 
   Y(t_1)\\ 
   \vdots \\
   Y(t_N)\\
 \end{bmatrix}x = \begin{bmatrix} 
   \tau(t_1)\\ 
   \vdots \\
   \tau(t_N)\\
 \end{bmatrix} - \begin{bmatrix} 
   {\sum_{l \in L} J_{l}^T f_l^{ext}}^1\\ 
   \vdots \\
   {\sum_{l \in L} J_{l}^T f_l^{ext}}^N\\
 \end{bmatrix} $$
$$ x = \begin{bmatrix} 
   Y(t_1)\\ 
   \vdots \\
   Y(t_N)\\
 \end{bmatrix}^{+}\begin{bmatrix} 
   \tau(t_1)\\ 
   \vdots \\
   \tau(t_N)\\
 \end{bmatrix} - \begin{bmatrix} 
   Y(t_1)\\ 
   \vdots \\
   Y(t_N)\\
 \end{bmatrix}^{+}\begin{bmatrix} 
   (\sum_{l \in L} J_{l}^T f_l^{ext})^1\\ 
   \vdots \\
   (\sum_{l \in L} J_{l}^T f_l^{ext})^N\\
 \end{bmatrix} $$
 
 The resulting matrices should have the following dimensions ($\text{DOFs}$ - Degrees of Freedom, $\text{NOLs}$ - Nr. of Links):

 $Y_1^N$: $N \cdot \text{DOFs}\times (10\cdot\text{NOLs})$
 
 $\tau^N_1$: $N \cdot \text{DOFs}$
 
 The equations only preduce sensible results if there are sufficient measurements and the measurements are reasonably noise free. Otherwise, a maximum-likehood estimator could be a good option which estimates the noise with known constant variance. Swevers describes one way of using an MLE and that it is robust against non-linear effects in the model such as modeled friction.

## Joint Excitation

A good estimation has physically consistent parameters (positive definite inertia values, respect the triangle inequality) and small errors between measured torques and simulated torques with newly estimated parameters, possibly limited to only a certain task. Siciliano recommends sufficiently rich trajectories that should however not excert unmodelled dynamic effects like elasticity.

In order to get meaningful sample data that allows good estimation, trajectories can be generated by optimization.
Swevers, Gensemann (1997) describe using Fourier series for all joints to get periodic excitation and using a common pulsation (i.e. frequency) so the excitation as a whole also stays periodic. This way it is supposedly easier to get time series measurements, averaging can be done, noise can be estimated and velocities and accelerations can be calculated analytically using the FFT to get near noise-free values. The joint angles for the $i$-th of $n$ joints are given by $N$ coefficients:

$$ q_i(t) = \sum_{l=1}^{N_i} \frac{a_l^i}{\omega_f l} \text{sin}(\omega_f lt)- \frac{b_l^i}{\omega_f l} \text{cos}(\omega_f lt) + q_{i0}$$

It is therefore parameterized by the Fourier Coefficients $a_l^i$ (amplitude of sin), $b_l^i$ (amplitude of cos) and $q_i0$ (offset on the position trajectory) for each term $l$ of each joint $i$.

Finding the right parameters is then an optimization problem to get ideal trajectories.
The "only correct" criterion for experimentation design is the covariance matrix of the estimated model parameters. 
Optimization takes the model constraints into account, too.

Ting, et. al describe a method based on Bayesian estimation that automatically detects noise and in a post-processing step, projects the estimate within consistency constraints (positive inertia values and parallel axis theorem). Higher measurement accuracy is claimed.

Ding, et. al. use an "artificial bee" algorithm to find the parameters for the fourier series parameters.

Alternatively, normal movement from existing behaviors can be tried to not end up with good inertia parameters for artificial use cases that still don't work well in actual behaviours.

## References

Swevers, Gansemann et. al.: Optimal Robot Excitation and Identification, 1997

Ting, Mistry, Peters, et. al.: A Bayesian Approach to Nonlinear Parameter Identification for Rigid Body Dynamics, 2006

Ding, Wu, et. al.: Dynamic Model Identification for 6-DOF Industrial Robots, 2015, http://www.hindawi.com/journals/jr/2015/471478/

Khosla: Parameter Identification of Robot Dynamics, 1985

Siciliano, Scaviccio, et. al.: Robotics Modelling, Planning and Control, Springer, 2009